<a href="https://colab.research.google.com/github/tackulus/204426_PROJECT/blob/main/Sprite_download.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pokebase

In [70]:
import io
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pokebase as pb
from tqdm import tqdm

In [72]:
sprite_img = []
for i in tqdm(range(1,899)):
    source = pb.SpriteResource('pokemon', i, official_artwork=True)
    binary = source.img_data
    img = plt.imread(io.BytesIO(binary))
    img = np.asarray(img)
    sprite_img.append(img)

sprite_img = np.asarray(sprite_img)
sprite_img.shape # pokedex n == index n-1

100%|██████████| 898/898 [01:23<00:00, 10.69it/s]


(898, 96, 96, 4)

In [74]:
np.save('sprite_img.npy', sprite_img)